<a href="https://colab.research.google.com/github/AndreiAf02/Job_Satisfaction_Burnout/blob/main/Job_Satisfaction_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import math
from scipy import stats

In [ ]:
def bootstrap_analysis(bootstrap_path_values, Working_condition_cols):
  mean_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  lbound_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  ubound_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  pbci_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)
  pbci_90_results = pd.DataFrame(columns = ['Path']+Working_condition_cols)

  for path in bootstrap_path_values['Path'].unique().tolist():
    data = bootstrap_path_values[bootstrap_path_values['Path']==path].reset_index(drop=True)
    mean_vals = [path]
    lbound_vals = [path]
    ubound_vals = [path]
    pbci_vals = [path]
    pbci_90_vals = [path]

    for col in Working_condition_cols:
      mean_vals.append(data[col].mean())
      ## Calculating the lower and upper bounds of the Percentile Bootstrap Confidence Interval (PBCI):
      lbound = np.percentile(data[col], 2.5)
      ubound = np.percentile(data[col], 97.5)
      if lbound > 0. or ubound < 0.:
        sig = ' *'
      else:
        sig = ''
      lbound_vals.append(lbound)
      ubound_vals.append(ubound)
      pbci_vals.append('['+str(round(lbound,4))+', '+str(round(ubound,4))+']'+sig)
      pbci_90_vals.append('['+str(round(np.percentile(data[col], 5.),4))+', '+str(round(np.percentile(data[col], 95),4))+']')

    mean_results.loc[len(mean_results)] = mean_vals
    lbound_results.loc[len(lbound_results)] = lbound_vals
    ubound_results.loc[len(ubound_results)] = ubound_vals
    pbci_results.loc[len(pbci_results)] = pbci_vals
    pbci_90_results.loc[len(pbci_90_results)] = pbci_90_vals



  return mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results

In [ ]:
def Bayesian_analysis(result_posterior_samples, factor_unique, dir_path_unique):
  path_ci = pd.DataFrame()
  path_prob = pd.DataFrame()

  path_ci['Factor'] = factor_unique
  path_prob['Factor'] = factor_unique

  med_ci_vals = []
  med_prob_vals = []
  mod_ci_vals = []
  mod_prob_vals = []
  dp_ci_vals = []
  dp_prob_vals = []

  for i in range(len(factor_unique)):

    col = factor_unique[i]
    ## Finding the Mediation Credibility Intervals and probabilities:
    data = result_posterior_samples[f'Indirect effect ({col})']
    lbound = np.percentile(data, 2.5)
    ubound = np.percentile(data, 97.5)
    probability_of_positivity = np.mean(data > 0)
    if lbound >0 or ubound < 0:
      sig = ' *'
    else:
      sig = ''
    med_ci_vals.append(f'[{lbound:.5f}, {ubound:.5f}]'+sig)
    med_prob_vals.append(probability_of_positivity)


    ## Finding the Moderation Credibility Intervals and probabilities:
    data_mod = result_posterior_samples[f'Moderation effect ({col})']
    lbound_mod = np.percentile(data_mod, 2.5)
    ubound_mod = np.percentile(data_mod, 97.5)
    probability_of_positivity_mod = np.mean(data_mod > 0)
    if lbound_mod >0 or ubound_mod < 0:
      sig_mod = ' *'
    else:
      sig_mod = ''
    mod_ci_vals.append(f'[{lbound_mod:.5f}, {ubound_mod:.5f}]'+sig_mod)
    mod_prob_vals.append(probability_of_positivity_mod)


    ## Finding the Direct Pathway Credibility Intervals and probabilities:
    data_dp = result_posterior_samples[dir_path_unique[i]]
    lbound_dp = np.percentile(data_dp, 2.5)
    ubound_dp = np.percentile(data_dp, 97.5)
    probability_of_positivity_dp = np.mean(data_dp > 0)
    if lbound_dp >0 or ubound_dp < 0:
      sig_dp = ' *'
    else:
      sig_dp = ''
    dp_ci_vals.append(f'[{lbound_dp:.5f}, {ubound_dp:.5f}]'+sig_dp)
    dp_prob_vals.append(probability_of_positivity_dp)


  path_ci['Mediation'] = med_ci_vals
  path_prob['Mediation'] = med_prob_vals
  path_ci['Moderation'] = mod_ci_vals
  path_prob['Moderation'] = mod_prob_vals
  path_ci['Direct Path'] = dp_ci_vals
  path_prob['Direct Path'] = dp_prob_vals

  return path_ci, path_prob

In [ ]:
def Bayesian_analysis_no_mod(result_posterior_samples, factor_unique, dir_path_unique):
  path_ci = pd.DataFrame()
  path_prob = pd.DataFrame()

  path_ci['Factor'] = factor_unique
  path_prob['Factor'] = factor_unique

  med_ci_vals = []
  med_prob_vals = []
  mod_ci_vals = []
  mod_prob_vals = []
  dp_ci_vals = []
  dp_prob_vals = []

  for i in range(len(factor_unique)):

    col = factor_unique[i]
    ## Finding the Mediation Credibility Intervals and probabilities:
    data = result_posterior_samples[f'Indirect effect ({col})']
    lbound = np.percentile(data, 2.5)
    ubound = np.percentile(data, 97.5)
    probability_of_positivity = np.mean(data > 0)
    if lbound >0 or ubound < 0:
      sig = ' *'
    else:
      sig = ''
    med_ci_vals.append(f'[{lbound:.5f}, {ubound:.5f}]'+sig)
    med_prob_vals.append(probability_of_positivity)


    ## Finding the Direct Pathway Credibility Intervals and probabilities:
    data_dp = result_posterior_samples[dir_path_unique[i]]
    lbound_dp = np.percentile(data_dp, 2.5)
    ubound_dp = np.percentile(data_dp, 97.5)
    probability_of_positivity_dp = np.mean(data_dp > 0)
    if lbound_dp >0 or ubound_dp < 0:
      sig_dp = ' *'
    else:
      sig_dp = ''
    dp_ci_vals.append(f'[{lbound_dp:.5f}, {ubound_dp:.5f}]'+sig_dp)
    dp_prob_vals.append(probability_of_positivity_dp)


  path_ci['Mediation'] = med_ci_vals
  path_prob['Mediation'] = med_prob_vals
  path_ci['Direct Path'] = dp_ci_vals
  path_prob['Direct Path'] = dp_prob_vals

  return path_ci, path_prob

## Czech Republic:
**Bootstrap:**

In [4]:
path_to_bootstrap = 'https://raw.githubusercontent.com/AndreiAf02/Job_Satisfaction_Burnout/refs/heads/main/Bootstrap/CZ/'

## Path values with no moderation:
bootstrap_vals_no_mod_pos_3 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_no_mod_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_4 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_no_mod_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_5 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_no_mod_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_6 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_no_mod_position_6.0.csv', sep=",", header=0).iloc[:,1:]

## Path values with moderation:
bootstrap_vals_pos_3 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_4 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_5 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_6 = pd.read_csv(path_to_bootstrap+'CZ_bootstrap_path_values_position_6.0.csv', sep=",", header=0).iloc[:,1:]




In [8]:
Working_condition_cols = ['Salary', 'Effort', 'Policy_influence', 'Teaching_percent',
                          'Research_percent', 'Academic_Resources', 'Perceived_Autonomy']


In [12]:
## Analyzing the bootstrap CI for the paths with age moderation:
i = 2
for bootstrap_vals in [bootstrap_vals_pos_3, bootstrap_vals_pos_4, bootstrap_vals_pos_5, bootstrap_vals_pos_6]:
  i = i+1
  print(f'95% CI for Position {i}:')
  mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results = bootstrap_analysis(bootstrap_vals, Working_condition_cols)
  print(pbci_results.set_index('Path').T)
  print('-'*50)


95% CI for Position 3:
Path                        Mediation         Moderation         Direct Path
Salary              [-0.0159, 0.0359]  [-0.0012, 0.0028]   [-0.0267, 0.1135]
Effort              [-0.0071, 0.0028]  [-0.0004, 0.0001]   [-0.0017, 0.0113]
Policy_influence    [-0.0505, 0.0549]  [-0.0033, 0.0054]   [-0.2053, 0.1001]
Teaching_percent    [-0.0028, 0.0055]  [-0.0002, 0.0004]     [-0.01, 0.0079]
Research_percent    [-0.0039, 0.0039]  [-0.0003, 0.0003]   [-0.0118, 0.0055]
Academic_Resources  [-0.0597, 0.0955]  [-0.0044, 0.0084]    [0.1287, 0.52] *
Perceived_Autonomy  [-0.0908, 0.0994]   [-0.009, 0.0049]  [0.1041, 0.5497] *
--------------------------------------------------
95% CI for Position 4:
Path                           Mediation         Moderation  \
Salary                 [-0.0035, 0.0051]  [-0.0007, 0.0003]   
Effort              [-0.0015, -0.0002] *     [-0.0, 0.0001]   
Policy_influence     [-0.0246, -0.003] *  [-0.0013, 0.0012]   
Teaching_percent       [-0.0013, 0.

In [13]:
## Analyzing the bootstrap CI for the paths without age moderation:
i = 2
for bootstrap_vals_no_mod in [bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                       bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
  i = i+1
  print(f'95% CI for Position {i}:')
  mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results = bootstrap_analysis(bootstrap_vals_no_mod, Working_condition_cols)
  print(pbci_results.set_index('Path').T)
  print('-'*50)

95% CI for Position 3:
Path                        Mediation         Direct Path
Salary              [-0.0071, 0.0135]   [-0.0267, 0.1135]
Effort              [-0.0028, 0.0011]   [-0.0017, 0.0113]
Policy_influence     [-0.042, 0.0184]   [-0.2054, 0.1001]
Teaching_percent     [-0.001, 0.0016]     [-0.01, 0.0079]
Research_percent     [-0.0018, 0.001]   [-0.0118, 0.0055]
Academic_Resources  [-0.0253, 0.0522]  [0.1288, 0.5197] *
Perceived_Autonomy  [-0.0225, 0.0695]   [0.104, 0.5497] *
--------------------------------------------------
95% CI for Position 4:
Path                           Mediation         Direct Path
Salary                 [-0.0023, 0.0062]  [0.0357, 0.0732] *
Effort              [-0.0018, -0.0004] *    [-0.0023, 0.002]
Policy_influence     [-0.0227, -0.002] *  [0.0203, 0.1042] *
Teaching_percent       [-0.0011, 0.0006]   [-0.0051, 0.0018]
Research_percent       [-0.0009, 0.0012]   [-0.0074, 0.0002]
Academic_Resources     [-0.0062, 0.0303]  [0.4627, 0.6163] *
Perceived_Au

**Bayesian:**

In [24]:
path_to_bayesian = 'https://raw.githubusercontent.com/AndreiAf02/Job_Satisfaction_Burnout/refs/heads/main/Bayesian/CZ/'

## Path values without moderation:
bayesian_vals_no_mod_pos_3 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_no_mod_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_4 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_no_mod_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_5 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_no_mod_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_6 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_no_mod_position_6.0.csv', sep=",", header=0).iloc[:,1:]

## Path values with moderation:
bayesian_vals_pos_3 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_4 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_5 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_6 = pd.read_csv(path_to_bayesian+'CZ_burnout_posterior_position_6.0.csv', sep=",", header=0).iloc[:,1:]

In [26]:
factor_unique = ['Salary', 'Effort', 'Teaching',
              'Research', 'Policy', 'Acad Resources', 'Autonomy']

dir_path_unique = ['dp1', 'dp2', 'dp3', 'dp4', 'dp5', 'dp6', 'dp7']


In [27]:
i = 2
for bayesian_vals in [bayesian_vals_pos_3, bayesian_vals_pos_4, bayesian_vals_pos_5, bayesian_vals_pos_6]:
  i = i+1
  print(f'95% CrI for Position {i}:')
  path_ci, path_prob = Bayesian_analysis(bayesian_vals, factor_unique, dir_path_unique)
  print(path_ci.set_index('Factor'))
  print('-'*50)

95% CrI for Position 3:
                          Mediation           Moderation           Direct Path
Factor                                                                        
Salary          [-0.01305, 0.03214]  [-0.00095, 0.00240]   [-0.00883, 0.10953]
Effort          [-0.00745, 0.00338]  [-0.00038, 0.00017]   [-0.00276, 0.01162]
Teaching        [-0.00194, 0.00481]  [-0.00015, 0.00035]   [-0.01009, 0.00789]
Research        [-0.00280, 0.00290]  [-0.00018, 0.00022]   [-0.01403, 0.00620]
Policy          [-0.03148, 0.04810]  [-0.00258, 0.00521]   [-0.20787, 0.09970]
Acad Resources  [-0.03980, 0.09835]  [-0.00296, 0.00716]  [0.11512, 0.53974] *
Autonomy        [-0.06225, 0.05281]  [-0.00734, 0.00289]  [0.15289, 0.54533] *
--------------------------------------------------
95% CrI for Position 4:
                             Mediation            Moderation  \
Factor                                                         
Salary             [-0.00434, 0.00508]   [-0.00083, 0.00025]  

In [32]:
i = 2
for bayesian_vals_no_mod in [bayesian_vals_no_mod_pos_3, bayesian_vals_no_mod_pos_4,
                             bayesian_vals_no_mod_pos_5, bayesian_vals_no_mod_pos_6]:
  i = i+1
  print(f'95% CrI for Position {i}:')
  path_ci, path_prob = Bayesian_analysis_no_mod(bayesian_vals_no_mod, factor_unique, dir_path_unique)
  print(path_ci.set_index('Factor'))
  print('-'*50)

95% CrI for Position 3:
                          Mediation           Direct Path
Factor                                                   
Salary          [-0.00828, 0.01575]   [-0.01237, 0.11104]
Effort          [-0.00337, 0.00136]   [-0.00267, 0.01235]
Teaching        [-0.00129, 0.00218]   [-0.01010, 0.00767]
Research        [-0.00233, 0.00173]   [-0.01249, 0.00582]
Policy          [-0.04645, 0.01901]   [-0.20690, 0.09658]
Acad Resources  [-0.03334, 0.04298]  [0.09797, 0.53165] *
Autonomy        [-0.02519, 0.07364]  [0.13963, 0.54675] *
--------------------------------------------------
95% CrI for Position 4:
                             Mediation           Direct Path
Factor                                                      
Salary             [-0.00244, 0.00607]  [0.03740, 0.07102] *
Effort          [-0.00174, -0.00043] *   [-0.00231, 0.00207]
Teaching           [-0.00114, 0.00052]   [-0.00506, 0.00165]
Research           [-0.00084, 0.00109]   [-0.00742, 0.00039]
Policy       

## Austria:
**Bootstrap:**

In [33]:
path_to_bootstrap = 'https://raw.githubusercontent.com/AndreiAf02/Job_Satisfaction_Burnout/refs/heads/main/Bootstrap/AT/'

## Path values with no moderation:
bootstrap_vals_no_mod_pos_2 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_2.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_3 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_4 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_5 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_no_mod_pos_6 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_no_mod_position_6.0.csv', sep=",", header=0).iloc[:,1:]

## Path values with moderation:
bootstrap_vals_pos_2 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_2.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_3 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_4 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_5 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bootstrap_vals_pos_6 = pd.read_csv(path_to_bootstrap+'AT_bootstrap_path_values_position_6.0.csv', sep=",", header=0).iloc[:,1:]




In [34]:
Working_condition_cols = ['Salary', 'Effort', 'Policy_influence', 'Teaching_percent',
                          'Research_percent', 'Academic_Resources', 'Perceived_Autonomy']


In [37]:
## Analyzing the bootstrap CI for the paths with age moderation:
i = 1
for bootstrap_vals in [bootstrap_vals_pos_2, bootstrap_vals_pos_3, bootstrap_vals_pos_4, bootstrap_vals_pos_5, bootstrap_vals_pos_6]:
  i = i+1
  print(f'95% CI for Position {i}:')
  mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results = bootstrap_analysis(bootstrap_vals, Working_condition_cols)
  print(pbci_results.set_index('Path').T)
  print('-'*50)


95% CI for Position 2:
Path                        Mediation         Moderation           Direct Path
Salary              [-0.0207, 0.0329]   [-0.001, 0.0007]      [-0.015, 0.0145]
Effort              [-0.0046, 0.0023]  [-0.0001, 0.0001]  [-0.0076, -0.0001] *
Policy_influence    [-0.1555, 0.0582]  [-0.0019, 0.0054]      [-0.025, 0.1887]
Teaching_percent    [-0.0111, 0.0041]  [-0.0001, 0.0004]     [-0.0031, 0.0098]
Research_percent    [-0.0065, 0.0039]  [-0.0001, 0.0002]     [-0.0001, 0.0087]
Academic_Resources  [-0.1172, 0.1577]   [-0.005, 0.0039]    [0.2012, 0.4553] *
Perceived_Autonomy  [-0.0481, 0.2348]  [-0.0064, 0.0015]    [0.2805, 0.5122] *
--------------------------------------------------
95% CI for Position 3:
Path                        Mediation         Moderation         Direct Path
Salary              [-0.0147, 0.0618]  [-0.0014, 0.0005]   [-0.0027, 0.0325]
Effort              [-0.0111, 0.0044]  [-0.0002, 0.0003]   [-0.0062, 0.0076]
Policy_influence    [-0.2995, 0.3346]  [

In [38]:
## Analyzing the bootstrap CI for the paths without age moderation:
i = 1
for bootstrap_vals_no_mod in [bootstrap_vals_no_mod_pos_2, bootstrap_vals_no_mod_pos_3, bootstrap_vals_no_mod_pos_4,
                       bootstrap_vals_no_mod_pos_5, bootstrap_vals_no_mod_pos_6]:
  i = i+1
  print(f'95% CI for Position {i}:')
  mean_results, lbound_results, ubound_results, pbci_results, pbci_90_results = bootstrap_analysis(bootstrap_vals_no_mod, Working_condition_cols)
  print(pbci_results.set_index('Path').T)
  print('-'*50)

95% CI for Position 2:
Path                        Mediation           Direct Path
Salary              [-0.0018, 0.0044]      [-0.015, 0.0145]
Effort              [-0.0009, 0.0003]  [-0.0076, -0.0001] *
Policy_influence    [-0.0071, 0.0225]      [-0.025, 0.1888]
Teaching_percent    [-0.0007, 0.0009]     [-0.0031, 0.0098]
Research_percent    [-0.0005, 0.0006]     [-0.0001, 0.0087]
Academic_Resources  [-0.0144, 0.0204]     [0.201, 0.4552] *
Perceived_Autonomy    [-0.02, 0.0547]    [0.2805, 0.5121] *
--------------------------------------------------
95% CI for Position 3:
Path                         Mediation         Direct Path
Salary              [0.0001, 0.0138] *   [-0.0027, 0.0325]
Effort               [-0.0028, 0.0003]   [-0.0062, 0.0076]
Policy_influence     [-0.0034, 0.0776]   [-0.1634, 0.1678]
Teaching_percent       [-0.002, 0.002]    [-0.006, 0.0108]
Research_percent     [-0.0006, 0.0027]   [-0.0029, 0.0107]
Academic_Resources   [-0.0055, 0.0987]  [0.0591, 0.4107] *
Perceived_

**Bayesian:**

In [39]:
path_to_bayesian = 'https://raw.githubusercontent.com/AndreiAf02/Job_Satisfaction_Burnout/refs/heads/main/Bayesian/AT/'

## Path values without moderation:
bayesian_vals_no_mod_pos_2 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_2.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_3 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_4 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_5 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_no_mod_pos_6 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_no_mod_position_6.0.csv', sep=",", header=0).iloc[:,1:]

## Path values with moderation:
bayesian_vals_pos_2 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_2.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_3 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_3.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_4 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_4.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_5 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_5.0.csv', sep=",", header=0).iloc[:,1:]
bayesian_vals_pos_6 = pd.read_csv(path_to_bayesian+'AT_burnout_posterior_position_6.0.csv', sep=",", header=0).iloc[:,1:]

In [40]:
factor_unique = ['Salary', 'Effort', 'Teaching',
              'Research', 'Policy', 'Acad Resources', 'Autonomy']

dir_path_unique = ['dp1', 'dp2', 'dp3', 'dp4', 'dp5', 'dp6', 'dp7']


In [41]:
i = 1
for bayesian_vals in [bayesian_vals_pos_2, bayesian_vals_pos_3, bayesian_vals_pos_4, bayesian_vals_pos_5, bayesian_vals_pos_6]:
  i = i+1
  print(f'95% CrI for Position {i}:')
  path_ci, path_prob = Bayesian_analysis(bayesian_vals, factor_unique, dir_path_unique)
  print(path_ci.set_index('Factor'))
  print('-'*50)

95% CrI for Position 2:
                          Mediation           Moderation  \
Factor                                                     
Salary          [-0.01477, 0.03895]  [-0.00108, 0.00044]   
Effort          [-0.00573, 0.00199]  [-0.00006, 0.00016]   
Teaching        [-0.01026, 0.00363]  [-0.00012, 0.00036]   
Research        [-0.00622, 0.00350]  [-0.00011, 0.00021]   
Policy          [-0.13868, 0.05746]  [-0.00181, 0.00486]   
Acad Resources  [-0.06374, 0.14882]  [-0.00454, 0.00214]   
Autonomy        [-0.05623, 0.16515]  [-0.00402, 0.00179]   

                           Direct Path  
Factor                                  
Salary             [-0.01313, 0.01884]  
Effort          [-0.00769, -0.00010] *  
Teaching           [-0.00345, 0.00992]  
Research           [-0.00036, 0.00876]  
Policy             [-0.01810, 0.18570]  
Acad Resources    [0.19909, 0.46145] *  
Autonomy          [0.27929, 0.50396] *  
--------------------------------------------------
95% CrI for Pos

In [42]:
i = 1
for bayesian_vals_no_mod in [bayesian_vals_no_mod_pos_2, bayesian_vals_no_mod_pos_3, bayesian_vals_no_mod_pos_4,
                             bayesian_vals_no_mod_pos_5, bayesian_vals_no_mod_pos_6]:
  i = i+1
  print(f'95% CrI for Position {i}:')
  path_ci, path_prob = Bayesian_analysis_no_mod(bayesian_vals_no_mod, factor_unique, dir_path_unique)
  print(path_ci.set_index('Factor'))
  print('-'*50)

95% CrI for Position 2:
                          Mediation             Direct Path
Factor                                                     
Salary          [-0.00142, 0.00436]     [-0.01258, 0.01935]
Effort          [-0.00097, 0.00031]  [-0.00798, -0.00028] *
Teaching        [-0.00076, 0.00080]     [-0.00271, 0.00977]
Research        [-0.00063, 0.00050]     [-0.00045, 0.00846]
Policy          [-0.00798, 0.02187]     [-0.01520, 0.18916]
Acad Resources  [-0.01180, 0.02161]    [0.19556, 0.45909] *
Autonomy        [-0.02402, 0.06292]    [0.27814, 0.50197] *
--------------------------------------------------
95% CrI for Position 3:
                           Mediation           Direct Path
Factor                                                    
Salary          [0.00042, 0.01377] *   [-0.00432, 0.03595]
Effort           [-0.00321, 0.00039]   [-0.00540, 0.00718]
Teaching         [-0.00207, 0.00213]   [-0.00516, 0.00944]
Research         [-0.00095, 0.00243]   [-0.00184, 0.00941]
Policy 